In [3]:
import os
import cv2
import numpy as np
import random

def combined_transform(image_1, image_2, max_trans, max_angle):
    rows, cols = image_1.shape[:2]

    # Random translation
    dx = random.uniform(-max_trans, max_trans) * cols
    dy = random.uniform(-max_trans, max_trans) * rows

    # Random rotation
    angle = random.uniform(-max_angle, max_angle)
    M_rotate = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

    # Adjust translation within the rotation matrix
    M_rotate[0, 2] += dx
    M_rotate[1, 2] += dy

    # Apply the transformations with no change in output size
    transformed_image_1 = cv2.warpAffine(image_1, M_rotate, (cols, rows), flags=cv2.INTER_NEAREST)
    transformed_image_2 = cv2.warpAffine(image_2, M_rotate, (cols, rows), flags=cv2.INTER_NEAREST)

    return transformed_image_1, transformed_image_2

def generate_augmented_images(input_folder, output_folder, num_augmentations=(0, 2), max_trans=0.1, max_angle=45):
    input_images = sorted(os.listdir(input_folder))
    output_images = sorted(os.listdir(output_folder))
    
    assert input_images == output_images, "Input and output folders must contain the same images with the same names."
    
    for img_name in input_images:
        input_img_path = os.path.join(input_folder, img_name)
        output_img_path = os.path.join(output_folder, img_name)

        input_image = cv2.imread(input_img_path, cv2.IMREAD_UNCHANGED)
        output_image = cv2.imread(output_img_path, cv2.IMREAD_UNCHANGED)

        for i in range(0, random.randint(num_augmentations[0], num_augmentations[1])):
            augmented_input, augmented_output = combined_transform(input_image, output_image, max_trans, max_angle)

            # Save augmented images
            cv2.imwrite(os.path.join(input_folder, f"{os.path.splitext(img_name)[0]}_aug{i}.png"), augmented_input)
            cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}_aug{i}.png"), augmented_output)


In [4]:
input_folder = '../images/images_train/input'
output_folder = '../images/images_train/output'
generate_augmented_images(input_folder, output_folder, (1, 2), 0.08, 35)